# Testing tac_obj

In [ ]:
from pytac import tac_obj

In [ ]:
tac_plan_a = """
#
# Line of filter for Plan_A
#
TAC_BLOCK Plan_A_TF0 DigitalTF 1.27608156e-07 -1.09201328e-01 2.01128208e-02 -2.20787340e-01 8.41941539e-02
TAC_BLOCK Plan_A_TF1 DigitalTF 1.00000000e+00 -7.35758822e-01 1.35335273e-01 -9.82584634e-01 2.79652134e-01 # My comment //color=red, style=dashed
TAC_BLOCK Plan_A_TF2 DigitalTF 1.00000000e+00 2.00000000e+00 1.00000000e+00 6.41074732e-01 -5.06380420e-03
TAC_BLOCK Plan_A_TF3 DigitalTF 1.00000000e+00 -1.21743565e+00 3.53285705e-01 -1.98930230e+00 9.89328869e-01
TAC_BLOCK Plan_A_TF4 DigitalTF 1.00000000e+00 -1.99097739e+00 9.90977393e-01 -1.99606523e+00 9.96080626e-01

TAC_LINK Plan_A_L0	Acc_M4 1	Plan_A_TF0	1
TAC_LINK Plan_A_L1	Plan_A_TF0 1	Plan_A_TF1	1
TAC_LINK Plan_A_L2	Plan_A_TF1 1	Plan_A_TF2	1 # // style=dashed, color=purple
TAC_LINK Plan_A_L3	Plan_A_TF2 1	Plan_A_TF3	1
TAC_LINK Plan_A_L4	Plan_A_TF3 1	Plan_A_TF4	1
TAC_LINK Plan_A_L5	Plan_A_TF4 1	Opl_Sign_M4	1

# End of filter Plan_A
#

"""

In [ ]:
tac_plan_a_graph = tac_obj(src=tac_plan_a.splitlines())
tac_plan_a_graph.graph
